In [181]:
# import os
# os.chdir('..')
# from src.modules import connect_lhl, test_models

In [182]:
conn = connect_lhl.lhl_delay_data('lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com','mid_term_project','lhl_student','lhl_student', 0)
conn.connection()
df = conn.sample_table('flights_test', samples='660556')

In [183]:
df.shape

(660556, 20)

In [184]:
df_flights_test = df

In [185]:
df_flights_test.shape

(660556, 20)

In [186]:
df_flights_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fl_date             660556 non-null  datetime64[ns]
 1   mkt_unique_carrier  660556 non-null  object        
 2   branded_code_share  660556 non-null  object        
 3   mkt_carrier         660556 non-null  object        
 4   mkt_carrier_fl_num  660556 non-null  int64         
 5   op_unique_carrier   660556 non-null  object        
 6   tail_num            659057 non-null  object        
 7   op_carrier_fl_num   660556 non-null  int64         
 8   origin_airport_id   660556 non-null  int64         
 9   origin              660556 non-null  object        
 10  origin_city_name    660556 non-null  object        
 11  dest_airport_id     660556 non-null  int64         
 12  dest                660556 non-null  object        
 13  dest_city_name      660556 no

In [187]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,"Spokane, WA",11298,DFW,"Dallas/Fort Worth, TX",1900-01-01 12:02:00,1734,N,212,1,1477
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,"Fort Smith, AR",11298,DFW,"Dallas/Fort Worth, TX",1900-01-01 14:25:00,1540,N,75,1,227
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,"Burlington, VT",11433,DTW,"Detroit, MI",1900-01-01 05:25:00,745,N,140,1,537
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,"Charlotte, NC",10821,BWI,"Baltimore, MD",1900-01-01 20:33:00,2159,N,86,1,361
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,"Minneapolis, MN",10529,BDL,"Hartford, CT",1900-01-01 20:30:00,6,N,156,1,1050


### Check for Nulls

In [188]:
df_flights_test.isnull().sum()

fl_date                  0
mkt_unique_carrier       0
branded_code_share       0
mkt_carrier              0
mkt_carrier_fl_num       0
op_unique_carrier        0
tail_num              1499
op_carrier_fl_num        0
origin_airport_id        0
origin                   0
origin_city_name         0
dest_airport_id          0
dest                     0
dest_city_name           0
crs_dep_time           104
crs_arr_time             0
dup                      0
crs_elapsed_time         0
flights                  0
distance                 0
dtype: int64

In [189]:
df_flights_test = df_flights_test.dropna()

In [190]:
df_flights_test.shape

(658953, 20)

## Apply Feature Transformation

In [191]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

### Departure TOD, Mean Arrival Delay by Flight/Route/TOD

In [192]:
df_flights_test['crs_dep_time'] = pd.to_datetime(df_flights_test['crs_dep_time'], format = '%H%M', errors = 'coerce')

C:\Users\User\AppData\Local\Temp\ipykernel_17136\2141607844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flights_test['crs_dep_time'] = pd.to_datetime(df_flights_test['crs_dep_time'], format = '%H%M', errors = 'coerce')


In [193]:
df_flights_test[df_flights_test['crs_dep_time'].isnull()]['crs_dep_time']

Series([], Name: crs_dep_time, dtype: datetime64[ns])

In [194]:
df_flights_test = df_flights_test[(df_flights_test['crs_dep_time'].isnull()==False)]

In [195]:
df_flights_test['crs_dep_time_hour'] = df_flights_test['crs_dep_time'].dt.hour

In [196]:
cut_labels_5 = ['early morning', 'late morning', 'midday', 'late afternoon', 'evening']
cut_bins = [0, 5, 10, 15, 20, 24]
df_flights_test['crs_dep_time_of_day'] = pd.cut(df_flights_test['crs_dep_time_hour'], bins=cut_bins, labels=cut_labels_5)

In [197]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,crs_dep_time_hour,crs_dep_time_of_day
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,DFW,"Dallas/Fort Worth, TX",1900-01-01 12:02:00,1734,N,212,1,1477,12,midday
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,DFW,"Dallas/Fort Worth, TX",1900-01-01 14:25:00,1540,N,75,1,227,14,midday
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,DTW,"Detroit, MI",1900-01-01 05:25:00,745,N,140,1,537,5,early morning
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,BWI,"Baltimore, MD",1900-01-01 20:33:00,2159,N,86,1,361,20,late afternoon
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,BDL,"Hartford, CT",1900-01-01 20:30:00,6,N,156,1,1050,20,late afternoon


In [198]:
# key1 = df_flights_test['op_unique_carrier'] + df_flights_test['op_carrier_fl_num'].astype(str) + df_flights_test['crs_dep_time_of_day'].astype(str) + df_flights_test['origin_city_name'] + df_flights_test['dest_city_name']
# key2 = df_flights_test['crs_dep_time_of_day'].astype(str)+ df_flights_test['origin_city_name']+ df_flights_test['dest_city_name']
# key3 = df_flights_test['crs_dep_time_of_day'].astype(str)

# df_flights_test['key1'] = key1
# df_flights_test['key2'] = key2
# df_flights_test['key3'] = key3

# for ind in df_flights_test.index:

#     if df_flights_test['key1'][ind] in dict1[0]:
            
#         df_flights_test['mean_flight_num_delay'] = df_flights_test['key1'].map(dict1[0])

#     elif df_flights_test['key2'][ind] in dict2[0]:

#         df_flights_test['mean_flight_num_delay'] = df_flights_test['key2'].map(dict2[0])

#     else:

#         df_flights_test['mean_flight_num_delay'] = df_flights_test['key3'].map(dict3[0])

In [199]:
# df_flights_test = df_flights_test.replace({'mean_flight_num_delay' : {np.nan: df_flights_test['key3'].map(dict3[0])}})

In [200]:
# g = sns.scatterplot(data=df_flights_test, x='mean_flight_num_delay', y='arr_delay')
# g.set_title('Arrival Delay by Mean Flight Number Delay - TEST SET')

In [201]:
# df_flights_test['mean_flight_num_delay'].corr(df_flights_test['arr_delay'])

### Mean Monthly Arrival Delay

In [202]:
# df_flights_test['fl_date'] = pd.to_datetime(df_flights_test['fl_date'], unit='ms')
df_flights_test['fl_date'] = pd.to_datetime(df_flights_test['fl_date'])
df_flights_test['year'] = df_flights_test['fl_date'].dt.year
df_flights_test['month'] = df_flights_test['fl_date'].dt.month
df_flights_test['day'] = df_flights_test['fl_date'].dt.day

In [203]:
# mean delay by month
month_dict = [{8: 10.135397157651207,
  6: 10.120094007050529,
  7: 8.901097631426921,
  5: 6.805443276744812,
  2: 6.5878846686997505,
  12: 6.068549859978083,
  4: 5.3825262618498595,
  3: 2.8932732468674436,
  10: 2.46615507960142,
  1: 2.2971539456662353,
  11: 2.2296219216094344,
  9: 0.5084617327607982}]

In [204]:
df_flights_test['mean_month_arr_delay'] = df_flights_test['month'].map(month_dict[0])

In [205]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dup,crs_elapsed_time,flights,distance,crs_dep_time_hour,crs_dep_time_of_day,year,month,day,mean_month_arr_delay
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,N,212,1,1477,12,midday,2020,1,24,2.297154
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,N,75,1,227,14,midday,2020,1,2,2.297154
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,N,140,1,537,5,early morning,2020,1,31,2.297154
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,N,86,1,361,20,late afternoon,2020,1,9,2.297154
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,N,156,1,1050,20,late afternoon,2020,1,26,2.297154


### Season

In [206]:
season_dict = {
    1:'Winter',
    2:'Winter',
    3:'Spring',
    4:'Spring',
    5:'Spring',
    6:'Summer',
    7:'Summer',
    8:'Summer',
    9:'Fall',
    10:'Fall',
    11:'Fall',
    12:'Winter'}

In [207]:
df_flights_test["season"] = df_flights_test["month"].map(season_dict)

In [208]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_elapsed_time,flights,distance,crs_dep_time_hour,crs_dep_time_of_day,year,month,day,mean_month_arr_delay,season
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,212,1,1477,12,midday,2020,1,24,2.297154,Winter
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,75,1,227,14,midday,2020,1,2,2.297154,Winter
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,140,1,537,5,early morning,2020,1,31,2.297154,Winter
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,86,1,361,20,late afternoon,2020,1,9,2.297154,Winter
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,156,1,1050,20,late afternoon,2020,1,26,2.297154,Winter


### Day of Week

In [209]:
df_flights_test['day_of_week'] = df_flights_test['fl_date'].dt.day_name()

In [210]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flights,distance,crs_dep_time_hour,crs_dep_time_of_day,year,month,day,mean_month_arr_delay,season,day_of_week
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,1,1477,12,midday,2020,1,24,2.297154,Winter,Friday
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,1,227,14,midday,2020,1,2,2.297154,Winter,Thursday
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,1,537,5,early morning,2020,1,31,2.297154,Winter,Friday
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,1,361,20,late afternoon,2020,1,9,2.297154,Winter,Thursday
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,1,1050,20,late afternoon,2020,1,26,2.297154,Winter,Sunday


### Weekday/Weekend and Arr_Delay

In [211]:
df_flights_test["is_weekend"] = df_flights_test['fl_date'].dt.dayofweek > 4

In [212]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,crs_dep_time_hour,crs_dep_time_of_day,year,month,day,mean_month_arr_delay,season,day_of_week,is_weekend
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,1477,12,midday,2020,1,24,2.297154,Winter,Friday,False
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,227,14,midday,2020,1,2,2.297154,Winter,Thursday,False
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,537,5,early morning,2020,1,31,2.297154,Winter,Friday,False
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,361,20,late afternoon,2020,1,9,2.297154,Winter,Thursday,False
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,1050,20,late afternoon,2020,1,26,2.297154,Winter,Sunday,True


### Holidays and Arr_Delay

In [213]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dr = pd.date_range(start='2018-01-01', end='2019-12-31')

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

df_flights_test['is_holiday'] = df_flights_test['fl_date'].isin(holidays)
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_dep_time_hour,crs_dep_time_of_day,year,month,day,mean_month_arr_delay,season,day_of_week,is_weekend,is_holiday
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,12,midday,2020,1,24,2.297154,Winter,Friday,False,False
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,14,midday,2020,1,2,2.297154,Winter,Thursday,False,False
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,5,early morning,2020,1,31,2.297154,Winter,Friday,False,False
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,20,late afternoon,2020,1,9,2.297154,Winter,Thursday,False,False
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,20,late afternoon,2020,1,26,2.297154,Winter,Sunday,True,False


### Region and Arr_Delay

In [214]:
new_origin = df_flights_test['origin_city_name'].str.split(",", n = 1, expand = True)
new_origin.head()

,0,1
0,Spokane,WA
1,Fort Smith,AR
2,Burlington,VT
3,Charlotte,NC
4,Minneapolis,MN


In [215]:
new_dest = df_flights_test['dest_city_name'].str.split(",", n = 1, expand = True)
new_dest.head()

,0,1
0,Dallas/Fort Worth,TX
1,Dallas/Fort Worth,TX
2,Detroit,MI
3,Baltimore,MD
4,Hartford,CT


In [216]:
df_flights_test['origin_state'] = new_origin[1].str.strip()
df_flights_test['dest_state'] = new_dest[1].str.strip()
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,year,month,day,mean_month_arr_delay,season,day_of_week,is_weekend,is_holiday,origin_state,dest_state
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,2020,1,24,2.297154,Winter,Friday,False,False,WA,TX
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,2020,1,2,2.297154,Winter,Thursday,False,False,AR,TX
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,2020,1,31,2.297154,Winter,Friday,False,False,VT,MI
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,2020,1,9,2.297154,Winter,Thursday,False,False,NC,MD
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,2020,1,26,2.297154,Winter,Sunday,True,False,MN,CT


In [217]:
# Add a region feature that groups states into four regions
# Region Dictionary for States *includes DC and Puerto Rico

state_region = {
    'WA': 'West',
    'OR': 'West',
    'CA': 'West',
    'AK': 'West',
    'HI': 'West',
    'ID': 'West',
    'MT': 'West',
    'WY': 'West',
    'NV': 'West',
    'UT': 'West',
    'CO': 'West',
    'AZ': 'West',
    'NM': 'West',
    'ND': 'Midwest',
    'SD': 'Midwest',
    'MN': 'Midwest',
    'NE': 'Midwest',
    'KS': 'Midwest',
    'IA': 'Midwest',
    'MO': 'Midwest',
    'WI': 'Midwest',
    'IL': 'Midwest',
    'MI': 'Midwest',
    'IN': 'Midwest',
    'OH': 'Midwest',
    'VT': 'Northeast',
    'ME': 'Northeast',
    'NH': 'Northeast',
    'MA': 'Northeast',
    'NY': 'Northeast',
    'PA': 'Northeast',
    'RI': 'Northeast',
    'CT': 'Northeast',
    'NJ': 'Northeast',
    'DE': 'South',
    'MD': 'South',
    'WV': 'South',
    'DC': 'South',
    'VA': 'South',
    'KY': 'South',
    'NC': 'South',
    'OK': 'South',
    'TX': 'South',
    'AR': 'South',
    'LA': 'South',
    'MS': 'South',
    'AL': 'South',
    'TN': 'South',
    'SC': 'South',
    'GA': 'South',
    'FL': 'South',
    'PR': 'Other',
    'VI': 'Other',
    'TT': 'Other'
}
len(state_region)

54

In [218]:
df_flights_test["origin_region"] = df_flights_test["origin_state"].map(state_region)

In [219]:
df_flights_test["dest_region"] = df_flights_test["dest_state"].map(state_region)

In [220]:
df_flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,day,mean_month_arr_delay,season,day_of_week,is_weekend,is_holiday,origin_state,dest_state,origin_region,dest_region
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,...,24,2.297154,Winter,Friday,False,False,WA,TX,West,South
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,...,2,2.297154,Winter,Thursday,False,False,AR,TX,South,South
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,...,31,2.297154,Winter,Friday,False,False,VT,MI,Northeast,Midwest
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,...,9,2.297154,Winter,Thursday,False,False,NC,MD,South,South
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,...,26,2.297154,Winter,Sunday,True,False,MN,CT,Midwest,Northeast


# Apply Data Transformation

### Check for Missing Values

In [221]:
sum(df_flights_test.isnull().sum())

0

### Fix DataTypes

In [222]:
df_flights_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658953 entries, 0 to 660555
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   fl_date               658953 non-null  datetime64[ns]
 1   mkt_unique_carrier    658953 non-null  object        
 2   branded_code_share    658953 non-null  object        
 3   mkt_carrier           658953 non-null  object        
 4   mkt_carrier_fl_num    658953 non-null  int64         
 5   op_unique_carrier     658953 non-null  object        
 6   tail_num              658953 non-null  object        
 7   op_carrier_fl_num     658953 non-null  int64         
 8   origin_airport_id     658953 non-null  int64         
 9   origin                658953 non-null  object        
 10  origin_city_name      658953 non-null  object        
 11  dest_airport_id       658953 non-null  int64         
 12  dest                  658953 non-null  object        
 13 

In [223]:
df_flights_test.shape

(658953, 34)

In [224]:
df_flights_test = df_flights_test[['crs_dep_time_of_day',
       'mean_month_arr_delay', 'season', 'day_of_week', 'is_weekend',
       'is_holiday', 'origin_region',
       'dest_region']]

### Transform Categorical Variables into Numeric Data

In [225]:
cat_feats_test = df_flights_test.dtypes[(df_flights_test.dtypes == 'object') | (df_flights_test.dtypes == 'category') ].index.tolist()
df_dummy_test = pd.get_dummies(df_flights_test[cat_feats_test])
df_dummy_test.head()

,crs_dep_time_of_day_early morning,crs_dep_time_of_day_late morning,crs_dep_time_of_day_midday,crs_dep_time_of_day_late afternoon,crs_dep_time_of_day_evening,season_Winter,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,...,origin_region_Midwest,origin_region_Northeast,origin_region_Other,origin_region_South,origin_region_West,dest_region_Midwest,dest_region_Northeast,dest_region_Other,dest_region_South,dest_region_West
0,0,0,1,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,1,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0


In [226]:
df_flights_test['season_Spring'] = 0
df_flights_test['season_Fall'] = 0
df_flights_test['season_Summer'] = 0

In [227]:
df_dummy_test.shape

(658953, 23)

In [228]:
df_flights_test.replace({
    "is_weekend" : {True : 1, False : 0}, 
    "is_holiday" : {True : 1, False : 0},
    }, inplace=True)

In [229]:
df_flights_test.shape

(658953, 11)

In [230]:
numeric_df_test = df_flights_test.drop(columns=cat_feats_test)

In [231]:
numeric_df_test.shape

(658953, 6)

In [232]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_data_test = pd.DataFrame(scaler.fit_transform(numeric_df_test), columns=numeric_df_test.columns)
scaled_data_test.head()

,mean_month_arr_delay,is_weekend,is_holiday,season_Spring,season_Fall,season_Summer
0,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0
1,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0
2,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0
3,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0
4,1.776357e-15,1.797091,0.0,0.0,0.0,0.0


In [233]:
scaled_data_test.shape

(658953, 6)

In [234]:
transformed_df_test = scaled_data_test.merge(df_dummy_test, left_index=True, right_index=True)
transformed_df_test.head()

,mean_month_arr_delay,is_weekend,is_holiday,season_Spring,season_Fall,season_Summer,crs_dep_time_of_day_early morning,crs_dep_time_of_day_late morning,crs_dep_time_of_day_midday,crs_dep_time_of_day_late afternoon,...,origin_region_Midwest,origin_region_Northeast,origin_region_Other,origin_region_South,origin_region_West,dest_region_Midwest,dest_region_Northeast,dest_region_Other,dest_region_South,dest_region_West
0,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
1,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1.776357e-15,-0.556455,0.0,0.0,0.0,0.0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,1.776357e-15,1.797091,0.0,0.0,0.0,0.0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0


In [235]:
sum(transformed_df_test.isnull().sum())

0

In [236]:
import pickle

test_predictions = transformed_df_test

with open(r'data\test_predictions.pickle', 'wb') as flight_data_file:
      pickle.dump(test_predictions, flight_data_file)

# Run Model

In [237]:
def IFPE_Regression(X_train, y_train, X_test, y_test, regressor, deg=None):
    
    ### INSTANTIATE THE MODEL

    ## LINEAR MODELS

    # ordinary least squares
    if regressor == 'linear':
        from sklearn.linear_model import LinearRegression
        reg = LinearRegression()
    
    # ridge regression
    elif regressor == 'ridge':
        from sklearn.linear_model import Ridge
        reg = Ridge(random_state=0)

    # lasso regression
    elif regressor == 'lasso':
        from sklearn.linear_model import Lasso
        reg = Lasso(random_state=0)

    # polynomial linear regression - need to add polynomial features first
    # https://www.geeksforgeeks.org/python-implementation-of-polynomial-regression/
    # https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions
    elif regressor == 'polynomial':
        from sklearn.linear_model import LinearRegression
        from sklearn.preprocessing import PolynomialFeatures
        poly = PolynomialFeatures(degree = deg)
        X_poly_train = poly.fit_transform(X_train)
        X_poly_test = poly.fit_transform(X_test)
        X_train = X_poly_train
        X_test = X_poly_test

        # poly.fit(X_poly, y_train)
        reg = LinearRegression()

    # stochastic gradient descent
    elif regressor == 'sgd':
        from sklearn.linear_model import SGDRegressor
        reg = SGDRegressor(random_state=0)
    
    ## SUPPORT VECTOR MACHINES

    # SVM regression https://scikit-learn.org/stable/modules/svm.html#regression
    elif regressor == 'svm':
        from sklearn import svm
        reg = svm.SVR()

    ## DECISION TREES

    # decision tree
    elif regressor == 'dt':
        from sklearn import tree
        reg = tree.DecisionTreeRegressor(random_state=0)

    ## ENSEMBLE METHODS

    # random forest
    elif regressor =='rf':
        from sklearn.ensemble import RandomForestRegressor
        reg = RandomForestRegressor(max_depth=2, random_state=0)

    # gradient boost
    elif regressor =='gb':
        from sklearn.ensemble import GradientBoostingRegressor
        reg = GradientBoostingRegressor(random_state=0)

    # voting regressor
    elif regressor =='vr':
        from sklearn.ensemble import GradientBoostingRegressor
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.linear_model import LinearRegression
        from sklearn.ensemble import VotingRegressor
        reg1 = GradientBoostingRegressor(random_state=1)
        reg2 = RandomForestRegressor(random_state=1)
        reg3 = LinearRegression()
        reg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])

    # xgboost
    elif regressor == 'xgb':
        import xgboost as xgb
        reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=0)



    ### FIT THE MODEL

    reg.fit(X_train, y_train)




    ### PREDICT THE RESPONSE FOR THE TEST DATASET

    y_pred = reg.predict(X_test)




    ### EVALUATE THE MODEL

    # Mean Absolute Error
    from sklearn.metrics import mean_absolute_error
    MAE = mean_absolute_error(y_test, y_pred)

    # Root Mean Square Error
    from sklearn.metrics import mean_squared_error
    RMSE = mean_squared_error(y_test, y_pred, squared=False)

    # r2
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test, y_pred)

    # Adjusted r2
    n = len(X_test)
    p = len(X_test[0,:])

    Adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

    ### COMPARE MODELS
    
    import pandas as pd
    model_comparison = pd.DataFrame({
        'Mean Absolute Error': MAE,
        'Root Mean Square Error': RMSE,
        'R Squared': r2,
        'Adjusted R Squared': Adj_r2
    }, index  = [str(regressor)])
    
    return model_comparison, reg

In [247]:
import pickle

with open(r'data\fe_flight_num_TRAINING_3_trans.pickle','rb') as flight_data_file:
     df_flights_TRAIN = pickle.load(flight_data_file)

In [248]:
df_flights_TRAIN.head()

,mean_month_arr_delay,is_weekend,is_holiday,arr_delay,crs_dep_time_of_day_early morning,crs_dep_time_of_day_late morning,crs_dep_time_of_day_midday,crs_dep_time_of_day_late afternoon,crs_dep_time_of_day_evening,season_Fall,...,origin_region_Midwest,origin_region_Northeast,origin_region_Other,origin_region_South,origin_region_West,dest_region_Midwest,dest_region_Northeast,dest_region_Other,dest_region_South,dest_region_West
0,0.240826,-0.604262,-0.157257,-0.407623,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,1.097784,-0.604262,-0.157257,-0.344193,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,-0.032054,-0.604262,-0.157257,-0.217333,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,-0.032054,-0.604262,-0.157257,-0.344193,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
4,-1.593737,-0.604262,-0.157257,-0.238476,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,1


In [249]:
import pickle

with open(r'data\fe_flight_num_TEST_3_trans.pickle','rb') as flight_data_file:
     df_flights_TEST = pickle.load(flight_data_file)

In [250]:
df_flights_TEST.head()

,mean_month_arr_delay,is_weekend,is_holiday,arr_delay,crs_dep_time_of_day_early morning,crs_dep_time_of_day_late morning,crs_dep_time_of_day_midday,crs_dep_time_of_day_late afternoon,crs_dep_time_of_day_evening,season_Fall,...,origin_region_Midwest,origin_region_Northeast,origin_region_Other,origin_region_South,origin_region_West,dest_region_Midwest,dest_region_Northeast,dest_region_Other,dest_region_South,dest_region_West
0,-0.032054,-0.604262,-0.157257,-0.069330,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0.554849,-0.604262,-0.157257,1.347269,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,1.097784,-0.604262,-0.157257,-0.196190,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.240826,-0.604262,-0.157257,-0.386479,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,-0.678107,1.654912,-0.157257,0.903261,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [251]:
X_train = df_flights_TRAIN.drop('arr_delay',axis=1).to_numpy()
X_test = df_flights_TEST.drop('arr_delay',axis=1).to_numpy()
y_train = df_flights_TRAIN['arr_delay'].to_numpy()
y_test = df_flights_TEST['arr_delay'].to_numpy()

In [252]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(74243, 29)
(24751, 29)
(74243,)
(24751,)


In [253]:
model_comparison_linear, reg = IFPE_Regression(X_train, y_train, X_test, y_test, regressor='linear', deg=None)
model_comparison_linear

,Mean Absolute Error,Root Mean Square Error,R Squared,Adjusted R Squared
linear,0.526212,1.097396,0.003203,0.002033


In [245]:
transformed_df_test.columns

Index(['mean_month_arr_delay', 'is_weekend', 'is_holiday', 'season_Spring',
       'season_Fall', 'season_Summer', 'crs_dep_time_of_day_early morning',
       'crs_dep_time_of_day_late morning', 'crs_dep_time_of_day_midday',
       'crs_dep_time_of_day_late afternoon', 'crs_dep_time_of_day_evening',
       'season_Winter', 'day_of_week_Friday', 'day_of_week_Monday',
       'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday',
       'day_of_week_Tuesday', 'day_of_week_Wednesday', 'origin_region_Midwest',
       'origin_region_Northeast', 'origin_region_Other', 'origin_region_South',
       'origin_region_West', 'dest_region_Midwest', 'dest_region_Northeast',
       'dest_region_Other', 'dest_region_South', 'dest_region_West'],
      dtype='object')

In [254]:
y_pred = reg.predict(transformed_df_test)

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [255]:
y_pred

array([-8.29814564e+10, -8.29814564e+10,  1.46484375e-02, ...,
        7.56835938e-03, -8.29814564e+10,  8.54492188e-03])

In [256]:
df_flights_test.head()

,crs_dep_time_of_day,mean_month_arr_delay,season,day_of_week,is_weekend,is_holiday,origin_region,dest_region,season_Spring,season_Fall,season_Summer
0,midday,2.297154,Winter,Friday,0,0,West,South,0,0,0
1,midday,2.297154,Winter,Thursday,0,0,South,South,0,0,0
2,early morning,2.297154,Winter,Friday,0,0,Northeast,Midwest,0,0,0
3,late afternoon,2.297154,Winter,Thursday,0,0,South,South,0,0,0
4,late afternoon,2.297154,Winter,Sunday,1,0,Midwest,Northeast,0,0,0


In [257]:
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-24,AA,AA_CODESHARE,AA,3648,MQ,N274NN,3648,11884,GEG,"Spokane, WA",11298,DFW,"Dallas/Fort Worth, TX",1900-01-01 12:02:00,1734,N,212,1,1477
1,2020-01-02,AA,AA_CODESHARE,AA,4186,MQ,N841AE,4186,11778,FSM,"Fort Smith, AR",11298,DFW,"Dallas/Fort Worth, TX",1900-01-01 14:25:00,1540,N,75,1,227
2,2020-01-31,DL,DL_CODESHARE,DL,4902,9E,N906XJ,4902,10785,BTV,"Burlington, VT",11433,DTW,"Detroit, MI",1900-01-01 05:25:00,745,N,140,1,537
3,2020-01-09,AA,AA,AA,1800,AA,N972UY,1800,11057,CLT,"Charlotte, NC",10821,BWI,"Baltimore, MD",1900-01-01 20:33:00,2159,N,86,1,361
4,2020-01-26,DL,DL,DL,1660,DL,N923AT,1660,13487,MSP,"Minneapolis, MN",10529,BDL,"Hartford, CT",1900-01-01 20:30:00,6,N,156,1,1050


In [258]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [259]:
submission = df[['fl_date', 'mkt_carrier',
       'mkt_carrier_fl_num', 'origin', 'dest']]

In [260]:
submission.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest
0,2020-01-24,AA,3648,GEG,DFW
1,2020-01-02,AA,4186,FSM,DFW
2,2020-01-31,DL,4902,BTV,DTW
3,2020-01-09,AA,1800,CLT,BWI
4,2020-01-26,DL,1660,MSP,BDL


In [261]:
import pandas as pd

In [262]:
submission = submission + y_pred

ValueError: Unable to coerce to Series, length must be 5: given 657356